In [1]:
import pandas as pd
import random
import os
import numpy as np
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
# import pickle5 as pickle

from sklearn.preprocessing import MinMaxScaler
from keras.wrappers.scikit_learn import KerasClassifier
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier


In [10]:
price = pd.read_csv('../price_universe.csv', index_col=0)
price.index = pd.to_datetime(price.index)

In [12]:
snp = price['S&P 500']
snp

2018-01-02    2323.76
2018-01-03    2339.29
2018-01-04    2350.30
2018-01-05    2366.48
2018-01-08    2370.14
               ...   
2021-12-23    4801.71
2021-12-27    4869.42
2021-12-28    4865.61
2021-12-29    4871.72
2021-12-30    4859.24
Name: S&P 500, Length: 956, dtype: float64

In [13]:
min_max_scaler = MinMaxScaler()
snp = min_max_scaler.fit_transform(snp.to_numpy().reshape(-1,1))

In [19]:
# snp = price.iloc[:, :1]
# train = snp[:-30]
# data_train = train.to_numpy()
# data_train

In [20]:
# test = snp[-30:]
# test
# data_test = test.to_numpy()
# data_test

In [21]:
# min_max_scaler = MinMaxScaler()
# data_train = min_max_scaler.fit_transform(train.to_numpy().reshape(-1,1))
# data_test = min_max_scaler.transform(test.to_numpy().reshape(-1,1))

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the Transformer Model
class StockTransformer(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_heads, ffn_hidden_size, dropout_rate):
        super().__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(input_size, num_heads, ffn_hidden_size, dropout_rate)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers)
        self.fc = nn.Linear(input_size, 1)

    def forward(self, x):
        x = self.transformer_encoder(x)
        x = self.fc(x)
        return x

# Initialize the model
input_size = n # n is the number of features in your dataset
hidden_size = 128
num_layers = 6
num_heads = 8
ffn_hidden_size = 128
dropout_rate = 0.1
model = StockTransformer(input_size, hidden_size, num_layers, num_heads, ffn_hidden_size, dropout_rate)

# Define the loss function and the optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    if epoch % 10 == 9:
        print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
        running_loss = 0.0

print('Finished Training')


In [27]:
# Import the required libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Transformer, Dense
from tensorflow.keras.models import Model

# Load the S&P 500 data into a pandas DataFrame
data = pd.read_csv('../price_universe.csv', index_col=0)

# Prepare the input data for the model
input_data = price['S&P 500']

# Define the Transformer model
input_layer = Input(shape=(input_data.shape[1],))
transformer = Transformer(num_heads=8, d_model=256, dff=1024, num_layers=2, activation="relu")(input_layer)
output_layer = Dense(units=1, activation="linear")(transformer)
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer="adam", loss="mean_squared_error")

# Train the model on the input data
model.fit(input_data, data["Close"], epochs=100, batch_size=32)

# Use the trained model to make predictions on new data
new_data = np.array([[100, 102, 98, 101]])
prediction = model.predict(new_data)
print("The predicted stock price for the new data is:", prediction[0][0])

ImportError: cannot import name 'Transformer' from 'tensorflow.keras.layers' (C:\Users\bongkyun\AppData\Roaming\Python\Python39\site-packages\keras\api\_v2\keras\layers\__init__.py)